# 3. Generating Tables and correlation filtering testing data

In [1]:
import napari
import pyclesperanto_prototype as cle
import apoc
import os
from skimage.io import imread, imsave
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from napari_skimage_regionprops import regionprops_table
from napari_simpleitk_image_processing import label_statistics

cle.select_device("Intel(R) Iris(TM) Plus Graphics 640")

<Intel(R) Iris(TM) Plus Graphics 640 on Platform: Apple (2 refs)>

In [2]:
image_folder = "/Users/mara/Desktop/Bachelorarbeit/Robert - PoL/20220523_training data/no training data/1. normalized images/"
image_raw_folder = "/Users/mara/Desktop/Bachelorarbeit/Robert - PoL/20220523_training data/no training data/1. images/"
labels_folder = "/Users/mara/Desktop/Bachelorarbeit/Robert - PoL/20220523_training data/no training data/3. in range result of object segmenter/"

In [3]:
file_list = sorted(os.listdir(image_folder))
file_list

['img04_20220429_ET358_56hpf_4.tif',
 'img07_20220513_ET358_56hpf_3.tif',
 'img09_20220513_ET358_56hpf_5.tif',
 'img11_20220513_ET358_56hpf_7.tif',
 'img16_20220520_hsp-mKate-DN-dynactin_56hpf_3.tif']

In [4]:
def analyze_image(filename):
    image = imread(image_folder + filename)
    image_raw = imread (image_raw_folder + filename)
    label = imread(labels_folder + filename)

    # measure in normalized image
    df_skimage = pd.DataFrame(regionprops_table(image , label, size = True, intensity = True, perimeter = True, shape = True))
    df_skimage["aspect_ratio"] = df_skimage["major_axis_length"]/df_skimage["minor_axis_length"]
    df_simpleitk = pd.DataFrame(label_statistics(image, label, size = True, intensity = True, perimeter = True, shape = True))
    DF = pd.merge(df_skimage, df_simpleitk, on = "label", suffixes = ('_skimage', '_simpleitk'))

    # measure in raw image
    df_skimage_raw = pd.DataFrame(regionprops_table(image_raw , label, size = True, intensity = True, perimeter = True, shape = True))
    df_skimage_raw["aspect_ratio"] = df_skimage_raw["major_axis_length"]/df_skimage_raw["minor_axis_length"]
    df_simpleitk_raw = pd.DataFrame(label_statistics(image_raw, label, size = True, intensity = True, perimeter = True, shape = True))
    DF_raw = pd.merge(df_skimage_raw, df_simpleitk_raw, on = "label", suffixes = ('_skimage', '_simpleitk'))

    #combine all
    BIG_DF = pd.merge(DF, DF_raw, on = "label", suffixes = ('_norm','_raw'))
    
    #add filename
    BIG_DF["filename"] = filename
    
    return BIG_DF

In [5]:
df_all_images = pd.DataFrame()

In [6]:
for i, filename in enumerate(file_list):
    print(filename)
    df_one_image = analyze_image(filename)
    df_all_images = pd.concat([df_all_images, df_one_image], ignore_index = True)
df_all_images

img04_20220429_ET358_56hpf_4.tif
img07_20220513_ET358_56hpf_3.tif
img09_20220513_ET358_56hpf_5.tif
img11_20220513_ET358_56hpf_7.tif
img16_20220520_hsp-mKate-DN-dynactin_56hpf_3.tif


,label,area_norm,bbox_area_norm,convex_area_norm,equivalent_diameter_norm,max_intensity_norm,mean_intensity_norm,min_intensity_norm,perimeter_skimage_norm,perimeter_crofton_norm,...,equivalent_ellipsoid_diameter_0_raw,equivalent_ellipsoid_diameter_1_raw,equivalent_spherical_perimeter_raw,equivalent_spherical_radius_raw,number_of_pixels_raw,number_of_pixels_on_border_raw,perimeter_simpleitk_raw,perimeter_on_border_raw,perimeter_on_border_ratio_raw,filename
0,1,149,208,158,13.773623,3.185185,1.397216,0.370370,57.556349,57.248358,...,7.612098,24.922525,43.271113,6.886812,149,26,57.248358,26.0,0.454161,img04_20220429_ET358_56hpf_4.tif
1,2,502,672,537,25.281738,7.370370,2.278663,0.296296,105.491378,102.889964,...,16.191692,39.474952,79.424921,12.640869,502,42,102.889964,42.0,0.408203,img04_20220429_ET358_56hpf_4.tif
2,3,2356,6150,2920,54.769995,9.444444,2.557961,0.259259,292.563492,280.441799,...,31.426042,95.454350,172.065014,27.384998,2356,0,280.441799,0.0,0.000000,img04_20220429_ET358_56hpf_4.tif
3,4,366,532,405,21.587165,4.777778,1.729609,0.296296,79.012193,77.589774,...,16.792796,27.750332,67.818078,10.793582,366,0,77.589774,0.0,0.000000,img04_20220429_ET358_56hpf_4.tif
4,5,60,80,62,8.740387,2.555556,1.225926,0.481481,26.727922,28.021176,...,7.359785,10.379974,27.458737,4.370194,60,0,28.021176,0.0,0.000000,img04_20220429_ET358_56hpf_4.tif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,26,219,475,321,16.698487,2.600000,0.930137,0.000000,85.805087,86.975083,...,14.468812,19.271760,52.459843,8.349243,219,0,86.975083,0.0,0.000000,img16_20220520_hsp-mKate-DN-dynactin_56hpf_3.tif
120,27,82,96,89,10.217908,2.600000,1.052439,0.100000,34.727922,35.605652,...,8.320613,12.547830,32.100505,5.108954,82,0,35.605652,0.0,0.000000,img16_20220520_hsp-mKate-DN-dynactin_56hpf_3.tif
121,28,314,594,372,19.994930,3.600000,1.242357,0.000000,91.219300,90.307245,...,13.419906,29.791358,62.815925,9.997465,314,0,90.307245,0.0,0.000000,img16_20220520_hsp-mKate-DN-dynactin_56hpf_3.tif
122,29,470,936,642,24.462677,3.800000,1.027660,0.100000,146.675144,140.431203,...,17.687945,33.832228,76.851768,12.231339,470,0,140.431203,0.0,0.000000,img16_20220520_hsp-mKate-DN-dynactin_56hpf_3.tif


In [7]:
df_all_images.to_csv("Testing_data_already_on_excluded_labels.csv")

# Filter after correlation

In [8]:
import pandas as pd
import numpy as np

In [9]:
excluded_labels = pd.read_csv("Testing_data_already_on_excluded_labels.csv")

In [10]:
excluded_labels.keys()

Index(['Unnamed: 0', 'label', 'area_norm', 'bbox_area_norm',
       'convex_area_norm', 'equivalent_diameter_norm', 'max_intensity_norm',
       'mean_intensity_norm', 'min_intensity_norm', 'perimeter_skimage_norm',
       'perimeter_crofton_norm', 'solidity_norm', 'extent_norm',
       'feret_diameter_max_norm', 'local_centroid-0_norm',
       'local_centroid-1_norm', 'major_axis_length_norm',
       'minor_axis_length_norm', 'orientation_norm', 'eccentricity_norm',
       'standard_deviation_intensity_norm', 'aspect_ratio_norm',
       'maximum_norm', 'mean_norm', 'median_norm', 'minimum_norm',
       'sigma_norm', 'sum_norm', 'variance_norm', 'elongation_norm',
       'feret_diameter_norm', 'flatness_norm', 'roundness_norm',
       'equivalent_ellipsoid_diameter_0_norm',
       'equivalent_ellipsoid_diameter_1_norm',
       'equivalent_spherical_perimeter_norm',
       'equivalent_spherical_radius_norm', 'number_of_pixels_norm',
       'number_of_pixels_on_border_norm', 'perimeter_s

In [11]:
to_drop = ['bbox_area_norm', 'convex_area_norm', 'equivalent_diameter_norm', 'perimeter_skimage_norm', 'perimeter_crofton_norm', 'feret_diameter_max_norm', 'major_axis_length_norm', 'standard_deviation_intensity_norm', 'maximum_norm', 'mean_norm', 'minimum_norm', 'sigma_norm', 'elongation_norm', 'feret_diameter_norm', 'flatness_norm', 'equivalent_ellipsoid_diameter_0_norm', 'equivalent_ellipsoid_diameter_1_norm', 'equivalent_spherical_perimeter_norm', 'equivalent_spherical_radius_norm', 'number_of_pixels_norm', 'perimeter_simpleitk_norm', 'perimeter_on_border_norm', 'area_raw', 'bbox_area_raw', 'convex_area_raw', 'equivalent_diameter_raw', 'perimeter_skimage_raw', 'perimeter_crofton_raw', 'solidity_raw', 'extent_raw', 'feret_diameter_max_raw', 'local_centroid-0_raw', 'local_centroid-1_raw', 'major_axis_length_raw', 'minor_axis_length_raw', 'orientation_raw', 'eccentricity_raw', 'aspect_ratio_raw', 'maximum_raw', 'mean_raw', 'median_raw', 'minimum_raw', 'sigma_raw', 'elongation_raw', 'feret_diameter_raw', 'flatness_raw', 'roundness_raw', 'equivalent_ellipsoid_diameter_0_raw', 'equivalent_ellipsoid_diameter_1_raw', 'equivalent_spherical_perimeter_raw', 'equivalent_spherical_radius_raw', 'number_of_pixels_raw', 'number_of_pixels_on_border_raw', 'perimeter_simpleitk_raw', 'perimeter_on_border_raw', 'perimeter_on_border_ratio_raw']

In [12]:
correlation_filtered = excluded_labels.drop(to_drop, axis=1)
correlation_filtered.columns

Index(['Unnamed: 0', 'label', 'area_norm', 'max_intensity_norm',
       'mean_intensity_norm', 'min_intensity_norm', 'solidity_norm',
       'extent_norm', 'local_centroid-0_norm', 'local_centroid-1_norm',
       'minor_axis_length_norm', 'orientation_norm', 'eccentricity_norm',
       'aspect_ratio_norm', 'median_norm', 'sum_norm', 'variance_norm',
       'roundness_norm', 'number_of_pixels_on_border_norm',
       'perimeter_on_border_ratio_norm', 'max_intensity_raw',
       'mean_intensity_raw', 'min_intensity_raw',
       'standard_deviation_intensity_raw', 'sum_raw', 'variance_raw',
       'filename'],
      dtype='object')

In [13]:
subset = ['label', 'area_norm', 'max_intensity_norm',
       'mean_intensity_norm', 'min_intensity_norm', 'solidity_norm',
       'extent_norm', 'local_centroid-0_norm', 'local_centroid-1_norm',
       'minor_axis_length_norm', 'orientation_norm', 'eccentricity_norm',
       'aspect_ratio_norm', 'median_norm', 'sum_norm', 'variance_norm',
       'roundness_norm', 'number_of_pixels_on_border_norm',
       'perimeter_on_border_ratio_norm', 'max_intensity_raw',
       'mean_intensity_raw', 'min_intensity_raw',
       'standard_deviation_intensity_raw', 'sum_raw', 'variance_raw',
       'filename']

subset_correlation_filtered=excluded_labels[subset]

In [14]:
subset_correlation_filtered.to_csv("Testingdata_Correlation_filtered_exclusion.csv")